In [ ]:
# run this at terminal first: export PYTHONPATH=/Library/Python/2.7/site-packages
# It's an issue with version of six that gets used

In [ ]:
import base64
import httplib2
from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials

cred_file = '/Users/damoncrockett/Desktop/stpete/stpete-cred.json'

DISC_URL='https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file)
service = discovery.build('vision', 'v1', credentials=creds, discoveryServiceUrl=DISC_URL)

import glob
import os
import pandas as pd

DIR = "/Users/damoncrockett/Desktop/stpete/sample/"

tmp = []
for file in glob.glob(os.path.join(DIR,"*.jpg")):
    tmp.append(file)
    
df = pd.DataFrame(tmp,columns=["local_path"])
n = len(df)

labels = []

In [ ]:
for i in range(n):
    with open(df.local_path.loc[i], 'rb') as image:
        image_content = base64.b64encode(image.read())
        service_request = service.images().annotate(body={
            'requests': [{
                'image': {
                    'content': image_content.decode('UTF-8')
                },
                'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 1000
                }]
            }]
        })
        
        response = service_request.execute()
        labels.append(response)

In [ ]:
len(labels)

In [ ]:
df.local_path.loc[0]

In [ ]:
labels[0]

In [ ]:
labels[1]

In [ ]:
num_labels = []

In [ ]:
for item in labels:
    if item['responses'][0]:
        num_labels.append(len(item['responses'][0]['labelAnnotations']))
    else:
        num_labels.append(0)

In [ ]:
from collections import Counter

In [ ]:
Counter(num_labels)

In [ ]:
for item in labels:
    if item['responses'][0]:
        print item['responses'][0]['labelAnnotations']
    else:
        print "0"

In [ ]:
# Ok here we go:

In [ ]:
tmp = []
for file in glob.glob(os.path.join(DIR,"*.jpg")):
    tmp.append(file)
    
df = pd.DataFrame(tmp,columns=["local_path"])
n = len(df)

labels = []

In [ ]:
df.info()

In [ ]:
breaks = [100,500,1000,5000,10000,15000,20000,25000,30000,35000,40000,45000]

In [ ]:
for i in range(n):
    
    try:
        with open(df.local_path.loc[i], 'rb') as image:
            image_content = base64.b64encode(image.read())
            service_request = service.images().annotate(body={
                'requests': [{
                    'image': {
                        'content': image_content.decode('UTF-8')
                    },
                    'features': [{
                        'type': 'LABEL_DETECTION',
                        'maxResults': 1000
                    }]
                }]
            })

            response = service_request.execute()
            labels.append(response['responses'][0]['labelAnnotations'])
            
    except:
        labels.append("nresp")
        
    if i in breaks:
        print i

In [ ]:
df['labels'] = labels
df.to_csv("./tmp.csv",index=False)

In [ ]:
df['labels'].loc[0]

In [ ]:
tag_score = []

for i in range(len(df)):
    label_list = df['labels'].loc[i]
    if label_list == "nresp":
        tag_score.append("nresp")
    else:
        tag_score_list = [item["description"]+":"+str(item["score"]) for item in label_list]
        tag_score.append(tag_score_list)

In [ ]:
df['tag_score'] = tag_score

In [ ]:
del df['labels']

In [ ]:
df.to_csv("./tmp_tmp.csv",index=False)

In [ ]:
tag_list = list(df.tag_score[df.tag_score!="nresp"])

In [ ]:
tag_list = [[item for sublist in tag_list for item in sublist]]

In [ ]:
tag_list = tag_list[0]

In [ ]:
tag_list = [item.split(":")[0] for item in tag_list]

In [ ]:
tag_set = list(set(tag_list))

In [ ]:
len(tag_set)

In [ ]:
tags = pd.DataFrame(tag_set,columns=['tag'])

In [ ]:
counts = []
for item in tags.tag:
    num_occur = tag_list.count(item)
    counts.append(num_occur)

tags['count'] = counts

In [ ]:
tags_sorted = tags.sort("count",ascending=False)

In [ ]:
tags_sorted.to_csv("./tags_counts.csv",index=False,encoding="utf-8")

In [ ]:
num_tags = []
for item in df.tag_score:
    if item=="nresp":
        num_tags.append(0)
    else:
        num_tags.append(len(item))

In [ ]:
max(num_tags)

In [ ]:
df